# Condor Game

The goal is to anticipate how asset prices will evolve by providing not a single forecasted value, but a **full probability distribution over future log-returns.**

## Probabilistic Forecasting

Probabilistic forecasting provides **a distribution of possible future values** rather than a single point estimate, allowing for uncertainty quantification. Instead of predicting only the most likely outcome, it estimates a range of potential outcomes along with their probabilities by outputting a **probability distribution**.

A probabilistic forecast models the conditional probability distribution of a future value $(Y_t)$ given past observations $(\mathcal{H}_{t-1})$. This can be expressed as:  

$$P(Y_t \mid \mathcal{H}_{t-1})$$

where $(\mathcal{H}_{t-1})$ represents the historical data up to time $(t-1)$. Instead of a single prediction $(\hat{Y}_t)$, the model estimates a full probability distribution $(f(Y_t \mid \mathcal{H}_{t-1}))$, which can take different parametric forms, such as a Gaussian:

$$Y_t \mid \mathcal{H}_{t-1} \sim \mathcal{N}(\mu_t, \sigma_t^2)$$

where $(\mu_t)$ is the predicted mean and $(\sigma_t^2)$ represents the uncertainty in the forecast.

Probabilistic forecasting can be handled through various approaches, including **variance forecasters**, **quantile forecasters**, **interval forecasters** or **distribution forecasters**, each capturing uncertainty differently.

In this notebook, we try to forecast the target location by a gaussian density function (or a mixture), the model output follows the form:

```python
[
    {
        "step": (k + 1) * step,
        "prediction": {
              "density": {
                            "name": "normal",
                            "params": {"loc": y_mean, "scale": y_var}
                          },
              "weight": weight
              }, ...
    }
    for k in range(0, horizon // step)
]
```

A **mixture density**, such as the gaussion mixture $\sum_{i=1}^{K} w_i \mathcal{N}(Y_t | \mu_i, \sigma_i^2)$ allows for capturing multi-modal distributions and approximate more complex distributions.

![proba_forecast_v3](https://github.com/Tarandro/image_broad/blob/main/proba_forecast_v3.png?raw=true)


**Probabilistic Forecasting** is particularly valuable in supply chain management. Below are some interesting resources for a deeper understanding:  

- [Probabilistic Forecasting](https://www.lokad.com/probabilistic-forecasting-definition/) – Overview of probabilistic forecasting and its applications.  
- [Quantile Forecasting](https://www.lokad.com/quantile-regression-time-series-definition/) – Explanation of quantile-based forecasting methods.  
- **Evaluation Metrics:**  
  - [Continuous Ranked Probability Score (CRPS)](https://www.lokad.com/continuous-ranked-probability-score/)  
  - [Cross-Entropy](https://www.lokad.com/cross-entropy-definition/)  
  - [Pinball Loss](https://www.lokad.com/pinball-loss-function-definition/)

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timezone, timedelta
from tqdm import tqdm

from condorgame.price_provider import shared_pricedb
from condorgame.tracker import TrackerBase
from condorgame.tracker_evaluator import TrackerEvaluator
from condorgame.examples.utils import load_test_prices_once, load_initial_price_histories_once
from condorgame.debug.plots import plot_quarantine, plot_prices, plot_return_prices, plot_scores

## What You Must Predict

Trackers must predict the **probability distribution of log-return price changes**, defined as:

$$
r_t = \log(P_t) - \log(P_{t-1})
$$

For each future step (e.g., +5 minutes, +10 minutes, …), your tracker must return a **probability density function (PDF)** describing where the **future log-return** is likely to be.

# Gaussian Step Tracker

A simple benchmark that predicts future log-returns by assuming they follow a Gaussian (normal) distribution estimated from recent historical data. It models the relative price change over each prediction step.

### **Key Ideas**  

- Historical prices are transformed into log-prices and predictions are made on log-returns.
- The tracker estimates:
    - Drift: mean historical log-return 𝜇
    - Volatility: standard deviation historical log-returns 𝜎
- For each future step 𝑘, it outputs a normal density:
$$\mathcal{r}_{t+k} \sim \mathcal{N}(\mu, \sigma)$$

Each density prediction must comply with the [density_pdf](https://github.com/microprediction/densitypdf/blob/main/densitypdf/__init__.py) specification.

In [2]:
class GaussianStepTracker(TrackerBase):
    """
    A benchmark tracker that models *future returns* as Gaussian-distributed.

    For each forecast step k, the tracker returns a normal distribution
    N(mu, sigma) where:
        - mu    = mean historical return
        - sigma = std historical return

    This is NOT a price-distribution; it is a distribution over returns
    between consecutive steps.
    """
    def __init__(self):
        super().__init__()

    def predict(self, asset: str, horizon: int, step: int):

        # Retrieve past prices with sampling resolution equal to the prediction step.
        pairs = self.prices.get_prices(asset, days=5, resolution=step)
        if not pairs:
            return []

        _, past_prices = zip(*pairs)

        if len(past_prices) < 3:
            return []

        # Compute historical returns
        returns = np.diff(past_prices)

        # Estimate drift (mean return) and volatility (std dev of returns)
        mu = float(np.mean(returns))
        sigma = float(np.std(returns))

        if sigma <= 0:
            return []

        num_segments = horizon // step

        # Produce one Gaussian for each future time step
        # The returned list must be compatible with the `density_pdf` library.
        distributions = []
        for k in range(1, num_segments + 1):
            distributions.append({
                "step": k * step,
                "type": "mixture",
                "components": [{
                    "density": {
                        "type": "builtin",             # Note: use 'builtin' instead of 'scipy' for speed
                        "name": "norm",  
                        "params": {"loc": mu, "scale": sigma}
                    },
                    "weight": 1
                }]
            })

        return distributions

## Configurations

In [3]:
##########
# For each asset and historical timestamp, compute a 24-hour density forecast 
# at 5-minute intervals and evaluate the tracker against actual outcomes.

assets = ["SOL", "BTC"]

# Prediction horizon = 24h (in seconds)
HORIZON = 86400
# Prediction step = 5 minutes (in seconds)
STEP = 300
# How often we evaluate the tracker (in seconds)
INTERVAL = 3600

# Base directory where all evaluation results will be stored
base_dir_results = "results"
os.makedirs(base_dir_results, exist_ok=True)

# End timestamp for the test data
# evaluation_end: datetime = datetime.now(timezone.utc)
evaluation_end: datetime = datetime(2025, 11, 15, 12, 00, 00, tzinfo=timezone.utc)

# Number of days of test data to load
days = 10
# Amount of warm-up history to load
days_history = 30

## Data

In [4]:
## Load the last N days of price data (test period)
test_asset_prices = load_test_prices_once(
    assets, shared_pricedb, evaluation_end, days=days
)

## Provide the tracker with initial historical data (for the first tick):
## load prices from the last H days up to N days ago
initial_histories = load_initial_price_histories_once(
    assets, shared_pricedb, evaluation_end, days_history=days_history, days_offset=days
)

## Run live simulation on historic data

In [5]:
# Setup tracker + evaluator
tracker_evaluator = TrackerEvaluator(GaussianStepTracker())

for asset, history_price in test_asset_prices.items():

    # First tick: initialize historical data
    tracker_evaluator.tick({asset: initial_histories[asset]})

    prev_ts = 0
    predict_count = 0
    pbar = tqdm(desc=f"Evaluating {asset}", smoothing=0.1)
    for ts, price in history_price:
        # Feed the new tick
        tracker_evaluator.tick({asset: [(ts, price)]})

        # Evaluate prediction every hour (ts is in second)
        if ts - prev_ts >= INTERVAL:
            prev_ts = ts
            pbar.update(1)
            predictions_evaluated = tracker_evaluator.predict(asset, HORIZON, STEP)

            # Periodically display results
            if predictions_evaluated and predict_count % 100 == 0:
                # print(f"My average crps score {asset}: {tracker_evaluator.overall_crps_score_asset(asset):.4f}")
                # print(f"My recent average crps score {asset}: {tracker_evaluator.recent_crps_score_asset(asset):.4f}")
                pbar.write(
                f"[{asset}] avg CRPS={tracker_evaluator.overall_crps_score_asset(asset):.4f} | "
                f"recent={tracker_evaluator.recent_crps_score_asset(asset):.4f}"
            )
            predict_count += 1
    pbar.close()
    print()

tracker_name = tracker_evaluator.tracker.__class__.__name__
print(f"\nTracker {tracker_name}:"
      f"\nFinal average crps score: {tracker_evaluator.overall_crps_score():.4f}")

current_results_dir = tracker_evaluator.to_json(horizon=HORIZON, step=STEP,
                                                interval=INTERVAL, base_dir=base_dir_results)

# Plot scoring timeline
timestamped_scores = tracker_evaluator.scores
plot_scores(timestamped_scores)

Evaluating SOL: 103it [00:11,  6.94it/s]

[SOL] avg CRPS=1.1691 | recent=1.1691


Evaluating SOL: 203it [00:26,  6.60it/s]

[SOL] avg CRPS=1.1244 | recent=1.0899


Evaluating SOL: 240it [00:32,  7.45it/s]


...

[BTC] avg CRPS=1.2716 | recent=1.2716


Evaluating BTC: 203it [00:26,  6.45it/s]

[BTC] avg CRPS=1.1659 | recent=1.0845


Evaluating BTC: 240it [00:32,  7.28it/s]




Tracker GaussianStepTracker:
Final average crps score: 1.1236
[✔] Tracker results saved to results\2025-11-06T12-00-00_to_2025-11-15T11-00-00\GaussianStepTracker_h86400_s300.json


In [6]:
## Density forecast over returns (for the last asset and last prediction)
plot_quarantine(asset, predictions_evaluated[0], tracker_evaluator.tracker.prices, mode="direct")

In [7]:
## Return forecast mapped into price space (for the last asset and last prediction)
print("CRPS score:", tracker_evaluator.scores[asset][-1][1])
plot_quarantine(asset, predictions_evaluated[0], tracker_evaluator.tracker.prices, mode="incremental")

CRPS score: 0.9412914246652291


# Tracker Comparison

In [8]:
from condorgame.examples.utils import load_all_results, plot_tracker_comparison

In [9]:
df_all = load_all_results(current_results_dir, horizon=HORIZON, step=STEP)
df_all

[✔] Found 3 files:
   - GaussianStepTracker_10stepshistoric_h86400_s300.json
   - GaussianStepTracker_h86400_s300.json
   - ShortGaussianStepTracker_h86400_s300.json


,tracker,asset,ts,score,time
0,GaussianStepTracker_10stepshistoric,SOL,1762430400,11.212902,2025-11-06 12:00:00+00:00
1,GaussianStepTracker_10stepshistoric,SOL,1762434000,8.332077,2025-11-06 13:00:00+00:00
2,GaussianStepTracker_10stepshistoric,SOL,1762437600,10.083480,2025-11-06 14:00:00+00:00
3,GaussianStepTracker_10stepshistoric,SOL,1762441200,3.040882,2025-11-06 15:00:00+00:00
4,GaussianStepTracker_10stepshistoric,SOL,1762444800,11.721770,2025-11-06 16:00:00+00:00
...,...,...,...,...,...
1291,ShortGaussianStepTracker,BTC,1763190000,4.192195,2025-11-15 07:00:00+00:00
1292,ShortGaussianStepTracker,BTC,1763193600,0.969780,2025-11-15 08:00:00+00:00
1293,ShortGaussianStepTracker,BTC,1763197200,3.030760,2025-11-15 09:00:00+00:00
1294,ShortGaussianStepTracker,BTC,1763200800,2.294930,2025-11-15 10:00:00+00:00


In [10]:
# Tracker comparison all assets
plot_tracker_comparison(df_all)

In [11]:
# Tracker comparison one assets
plot_tracker_comparison(df_all, 'SOL')

In [12]:
# Tracker comparison one assets
plot_tracker_comparison(df_all, 'BTC')